<a href="https://colab.research.google.com/github/franklinscudder/DiffieHellmanExample/blob/main/DiffieHellman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip3 install numba --upgrade

Requirement already up-to-date: numba in /usr/local/lib/python3.7/dist-packages (0.53.1)


In [25]:
from numba import njit
import random
from math import gcd as bltin_gcd

def primRoots(modulo):
    required_set = {num for num in range(1, modulo) if bltin_gcd(num, modulo) }
    return [g for g in range(1, modulo) if required_set == {pow(g, powers, modulo) for powers in range(1, modulo)}]

@njit
def Prime(n):
    if n & 1 == 0:
        return 2
    d= 3
    while d * d <= n:
        if n % d == 0:
            return d
        d= d + 2
    return 0

@njit
def getPrime(bits):
    p=1
    n = 0
    bits = 2 ** bits
    base = random.randint(bits/2, bits)

    while p is not 0:
        n+=1
        p = Prime(base + n)

    return base + n

class user:
    def __init__(self):
        self.rx = []
        self.secret = random.randint(5,128)
    
    def initDH(self, target, bits):
        print(f"Finding a {bits}-bit prime...")
        self.p = getPrime(bits)
        print(f"Finding a suitable generator...")
        self.g = random.choice(primRoots(self.p))
        print(f"The shared prime is: {self.p}")
        print(f"The shared generator is: {self.g}")        
        self.myPubKey = (self.g ** self.secret) % self.p
        print(f"The initialiser's public key is: {self.myPubKey}")
        target.rx.append(self.p)
        target.rx.append(self.g)
        target.rx.append(self.myPubKey)
        target.respDH(self)
        self.theirPubKey = self.rx.pop()
        print(f"The respondant's public key is: {self.theirPubKey}")
        self.finalKey = (self.theirPubKey ** self.secret) % self.p
        print(f"The final agreed private key is: {self.finalKey}")

    def respDH(self, target):
        self.theirPubKey = self.rx.pop()
        self.g = self.rx.pop()
        self.p = self.rx.pop()
        self.myPubKey = (self.g ** self.secret) % self.p
        target.rx.append(self.myPubKey)
        self.finalKey = (self.theirPubKey ** self.secret) % self.p

alice = user()
bob = user()
alice.initDH(bob, 12)


print(alice.finalKey)
print(bob.finalKey)

Finding a 12-bit prime...
Finding a suitable generator...
The shared prime is: 3833
The shared generator is: 492
The initialiser's public key is: 1112
The respondant's public key is: 3344
The final agreed private key is: 3035
3035
3035
